In [6]:
#!pip install mxnet
#!pip install gluonnlp pandas tqdm
#!pip install sentencepiece
#!pip install transformers
#!pip install torch

In [7]:
#!pip install git+https://git@github.com/SKTBrain/KoBERT.git@master

In [8]:
#from google.colab import drive
#drive.mount('/content/drive')

In [9]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm, tqdm_notebook

In [10]:
#kobert
from kobert.utils import get_tokenizer
from kobert.pytorch_kobert import get_pytorch_kobert_model

#transformers
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

In [11]:
device = torch.device("cuda:0")

In [12]:
bertmodel, vocab = get_pytorch_kobert_model()

using cached model. /home/piai/test/Project_NLP/Model_training_menu/.cache/kobert_v1.zip
using cached model. /home/piai/test/Project_NLP/Model_training_menu/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


In [13]:
#!pip install openpyxl

In [14]:
import pandas as pd
chatbot_data = pd.read_excel('데이터셋3차.xlsx')

In [15]:
chatbot_data.sample(n = 10)

,input,type1
66,양고기스테이크 양고기스테이크,양고기스테이크
99,인기 메뉴로 주세요,인기
25,가츠동,가츠동
15,저는 비빔밥이요,비빔밥
61,양고기스테이크 줄래?,양고기스테이크
90,제일 인기 있는 메뉴가 뭐예요?,인기
135,아무거나 좋아,아무거나
67,음 양고기스테이크,양고기스테이크
77,라면은?,라면
6,비빔밥으로 주문할게요,비빔밥


In [16]:
chatbot_data.loc[(chatbot_data['type1'] == "비빔밥"), 'type1'] = 0  
chatbot_data.loc[(chatbot_data['type1'] == "가츠동"), 'type1'] = 1 
chatbot_data.loc[(chatbot_data['type1'] == "양고기스테이크"), 'type1'] = 2  
chatbot_data.loc[(chatbot_data['type1'] == "라면"), 'type1'] = 3 
chatbot_data.loc[(chatbot_data['type1'] == "인기"), 'type1'] = 4 
chatbot_data.loc[(chatbot_data['type1'] == "아무거나"), 'type1'] = 5 


data_list = []
for q, label in zip(chatbot_data['input'], chatbot_data['type1'])  :
    data = []
    data.append(q)
    data.append(str(label))

    data_list.append(data)

In [17]:
from sklearn.model_selection import train_test_split
                                                         
dataset_train, dataset_test = train_test_split(data_list, test_size=0.25, random_state=0)
#print(len(dataset_train))
#print(len(dataset_test))


In [18]:
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)

        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))

In [19]:
# Setting parameters
max_len = 64
batch_size = 10
warmup_ratio = 0.1
num_epochs = 10
max_grad_norm = 1
log_interval = 200
learning_rate =  5e-5

In [20]:
#토큰화
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

data_train = BERTDataset(dataset_train, 0, 1, tok, max_len, True, False)
data_test = BERTDataset(dataset_test, 0, 1, tok, max_len, True, False)

using cached model. /home/piai/test/Project_NLP/Model_training_menu/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


In [21]:
train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, num_workers=5)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, num_workers=5)

In [22]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=7,   ##클래스 수 조정##
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

In [53]:
#BERT 모델 불러오기
model = BERTClassifier(bertmodel,  dr_rate=0.5).to(device)

#optimizer와 schedule 설정
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()

t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

#정확도 측정을 위한 함수 정의
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc
    
train_dataloader

In [54]:
model

BERTClassifier(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(8002, 768, padding_idx=1)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True

In [55]:
for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    model.train()
    
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
    
    model.eval()
    with torch.no_grad():
        for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):
            token_ids = token_ids.long().to(device)
            segment_ids = segment_ids.long().to(device)
            valid_length= valid_length
            label = label.long().to(device)
            out = model(token_ids, valid_length, segment_ids)
            test_acc += calc_accuracy(out, label)
        print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))

/tmp/ipykernel_5340/2306323563.py:6: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):


  0%|          | 0/11 [00:00<?, ?it/s]

epoch 1 batch id 1 loss 1.9045318365097046 train acc 0.1
epoch 1 train acc 0.4636363636363636


/tmp/ipykernel_5340/2306323563.py:25: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):


  0%|          | 0/4 [00:00<?, ?it/s]

...

epoch 9 test acc 1.0


  0%|          | 0/11 [00:00<?, ?it/s]

epoch 10 batch id 1 loss 0.07189915329217911 train acc 1.0
epoch 10 train acc 1.0


  0%|          | 0/4 [00:00<?, ?it/s]

epoch 10 test acc 1.0


In [85]:
#토큰화
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

def predict(predict_sentence, menu_c): #menu_c는 메뉴 list

    data = [predict_sentence, '0']
    dataset_another = [data]

    another_test = BERTDataset(dataset_another, 0, 1, tok, max_len, True, False)
    test_dataloader = torch.utils.data.DataLoader(another_test, batch_size=batch_size, num_workers=5)
    
    model.eval()
    with torch.no_grad():
        for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(test_dataloader):
            token_ids = token_ids.long().to(device)
            segment_ids = segment_ids.long().to(device)

            valid_length= valid_length
            label = label.long().to(device)
            out = model(token_ids, valid_length, segment_ids)

            test_eval=[]
            
            logits = out
            logits = logits.detach().cpu().numpy()
            #print(logits)
            if np.argmax(logits) == 0:
                menu_c['k'] -= 1
                test_eval.append("비빔밥, 한식은 4층을 열어주세요.")
            elif np.argmax(logits) == 1:
                menu_c['j'] -= 1
                test_eval.append("가츠동, 일식은 3층을 열어주세요.")
            elif np.argmax(logits) == 2:
                menu_c['a'] -= 1
                test_eval.append("양고기스테이크, 양식은 2층을 열어주세요.")
            elif np.argmax(logits) == 3:
                test_eval.append("라면, 승무원을 불러드리겠습니다.")
            elif np.argmax(logits) == 4: #인기
                random_menu = min(menu_c['k'], menu_c['j'], menu_c['a'])
                #result에 그 key값 넣기
                reverse_dict= dict(map(reversed, menu_c.items()))
                result = reverse_dict[random_menu]
                menu_c[result] -= 1
                if result == 'k':
                    test_eval.append("비빔밥, 한식은 4층을 열어주세요.")
                elif result == 'j':
                    test_eval.append("가츠동, 일식은 3층을 열어주세요.")
                elif result == 'a':
                    test_eval.append("양고기스테이크, 양식은 2층을 열어주세요.")
                        
            elif np.argmax(logits) == 5: #아무거나
                #3가지 메뉴 중 가장 많이 남은 것 줌
                random_menu = max(menu_c['k'], menu_c['j'], menu_c['a'])
                #result에 그 key값 넣기
                reverse_dict= dict(map(reversed, menu_c.items()))
                result = reverse_dict[random_menu]
                menu_c[result] -= 1
                if result == 'k':
                    test_eval.append("비빔밥, 한식은 4층을 열어주세요.")
                elif result == 'j':
                    test_eval.append("가츠동, 일식은 3층을 열어주세요.")
                elif result == 'a':
                    test_eval.append("양고기스테이크, 양식은 2층을 열어주세요.")
              
        
            if not test_eval:
                print("죄송합니다 고객님, 다시 한 번 말씀해주세요")
                return False
            print(">> 말씀하신 메뉴는 " + test_eval[0] )
            return True

using cached model. /home/piai/test/Project_NLP/Model_training_menu/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


In [89]:
menu_c = {'k': 110, 'j': 130, 'a': 130}

In [90]:
torch.save(model.state_dict(), 'model_menu.pth')
#torch.save(model, '/home/piai/test/해보자/모델학습/model_menu.pth')

In [91]:
#질문 무한반복하기! 0 입력시 종료

while True :
    sentence = input("하고싶은 말을 입력해주세요 : ")
    if sentence == '0' :
        break
    predict(sentence, menu_c)
    print(menu_c)
    print("\n")
    


하고싶은 말을 입력해주세요 : 양고기주세요!
>> 말씀하신 메뉴는 양고기스테이크, 양식은 2층을 열어주세요.
{'k': 110, 'j': 130, 'a': 129}


하고싶은 말을 입력해주세요 : 스테이크주세요~
>> 말씀하신 메뉴는 양고기스테이크, 양식은 2층을 열어주세요.
{'k': 110, 'j': 130, 'a': 128}


하고싶은 말을 입력해주세요 : 가츠동 주세요!
>> 말씀하신 메뉴는 가츠동, 일식은 3층을 열어주세요.
{'k': 110, 'j': 129, 'a': 128}


하고싶은 말을 입력해주세요 : 저는 비빔밥주세요~
>> 말씀하신 메뉴는 비빔밥, 한식은 4층을 열어주세요.
{'k': 109, 'j': 129, 'a': 128}


하고싶은 말을 입력해주세요 : 인기있는 거 주세요!
k
>> 말씀하신 메뉴는 비빔밥, 한식은 4층을 열어주세요.
{'k': 108, 'j': 129, 'a': 128}


하고싶은 말을 입력해주세요 : 아무거나 주세요~
j
>> 말씀하신 메뉴는 가츠동, 일식은 3층을 열어주세요.
{'k': 108, 'j': 128, 'a': 128}




KeyboardInterrupt: Interrupted by user

In [84]:
menu_c

{'k': 109, 'j': 130, 'a': 130}